# Clustering de Zonas Geológicas

El clustering de zonas geológicas es una técnica de agrupamiento que permite identificar patrones y similitudes entre diferentes intervalos geológicos dentro de un pozo o entre múltiples pozos. Estas agrupaciones son útiles para entender el comportamiento del yacimiento, identificar zonas con características similares y optimizar estrategias de desarrollo, como la selección de fluidos de inyección o el diseño de estimulaciones.

El análisis considera variables clave como la **permeabilidad**, **porosidad** y el **diámetro de garganta de poro (R35)**, que se calcula a partir de la permeabilidad y porosidad mediante una relación empírica. Estas características geológicas se analizan para identificar clusters que representan diferentes zonas del yacimiento.


## Contenido
- [1 - Introducción](#1)
  - [1.1 Objetivo](#1.1)
- [2 - Imports Necesarios](#2)
- [3 - Exploración](#3)
  - [3.1 Datos](#3.1)
  - [3.2 Visualización](#3.2)
- [4 - Funciones Auxiliares](#4)
  - [4.1 - Cálculo del R35](#4.1)
  - [4.2 - Función de Escalado](#4.2)
- [5 - Clustering K-Means](#5)
- [6 - Clustering DBSCAN](#6)
- [7 - Evaluación del Clustering](#7)
  - [7.1 Visualización](#7.1)
  - [7.2 Métricas de Evaluación](#7.2)


<a name="1"></a>

## 1 - Introducción 
En este notebook, exploraremos el uso de técnicas de clustering para identificar patrones y similitudes en zonas geológicas de pozos petroleros. El análisis se centrará en agrupar zonas con características geológicas similares utilizando variables clave como:
1. Permeabilidad (mD).
2. Porosidad (%).
3. Diámetro de garganta de poro (R35).

Estas técnicas permiten identificar grupos homogéneos dentro de un yacimiento, lo que facilita la toma de decisiones para optimizar estrategias de inyección, estimulación y desarrollo del campo.

<a name="1.1"></a>

### 1.1 Objetivo

El objetivo de este estudio es aplicar algoritmos de clustering para identificar y analizar zonas geológicas con comportamientos similares dentro de uno o varios pozos. Usaremos datos históricos de mediciones geológicas (permeabilidad, porosidad y R35) para descubrir patrones que apoyen la planificación y optimización de las operaciones. Este análisis se realizará sobre un conjunto de datos estructurado en formato csv.


<a name="2"></a>
## 2 - Imports Necesarios

El siguiente bloque de código importará las bibliotecas necesarias para ejecutar este notebook..

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'notebook'

<a name="2"></a>
## 3 - Exploración


<a name="3.1"></a>

### 3.1 Datos

Los datos utilizados en este análisis están almacenados en un archivo CSV y contienen las siguientes Columnas:
* depth
* permeability (mD)
* porosity (%)

Se encuentran valores de permeabilidad y porosidad cada 0.5 ft

In [8]:
df=pd.read_csv('garganta_poro.csv')

In [10]:
#Se revisan los primeros 5 registros
df.head()

,depth,permeability,porosity,Pozo
0,7159.5,0.0,0.0,well1
1,7160.0,0.0,0.0,well1
2,7160.5,0.0,0.0,well1
3,7161.0,0.0,0.0,well1
4,7161.5,0.0,0.0,well1
